
---------------
#           Mini Projet: Data Processing - Data Viz   
---------------

### importation des  nécessaires 

In [1]:
# importation des bibliothèques nécessaires
import pandas as pd
import numpy as np
import requests
import zipfile
import os
from io import BytesIO
import openpyxl
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
import pandas as pd

### Initialisation et téléchargement des données

In [2]:


# 1. Identification du code ISO du Burkina Faso: le code ISO du Burkina Faso est 'BF'


# 2. Téléchargement du fichier de données
print("\n2. TÉLÉCHARGEMENT DES DONNÉES")
print("-" * 30)

# 3. Lien du fichier zip pour le Burkina Faso
url = f"https://download.geonames.org/export/dump/BF.zip"

# Téléchargement du fichier
response = requests.get(url)
response.raise_for_status()

# Extraction du fichier zip
with zipfile.ZipFile(BytesIO(response.content)) as zip_file:
    # Liste des fichiers dans le zip
    file_list = zip_file.namelist()
    print(f"Fichiers dans le zip: {file_list}")
        
    # Extraction du fichier principal ( BF.txt)
    main_file = "BF.txt"
    if main_file in file_list:
        zip_file.extract(main_file, ".")
        print(f"✓ Fichier {main_file} extrait")
    else:
        print(f"Fichier {main_file} non trouvé dans le zip")
            



2. TÉLÉCHARGEMENT DES DONNÉES
------------------------------
Fichiers dans le zip: ['readme.txt', 'BF.txt']
✓ Fichier BF.txt extrait


In [3]:

# 3. Prétraitement des données

# Définition des noms de colonnes selon la documentation GeoNames
column_names = [
    'geonameid', 'name', 'asciiname', 'alternatenames', 'latitude', 'longitude',
    'feature_class', 'feature_code', 'country_code', 'cc2', 'admin1_code',
    'admin2_code', 'admin3_code', 'admin4_code', 'population', 'elevation',
    'dem', 'timezone', 'modification_date'
]

# Lecture du fichier avec les noms de colonnes appropriés
df = pd.read_csv("BF.txt", sep='\t', names=column_names, encoding='utf-8')
print("\n **********************************")
print(f" Nombre de lignes (lieux): {len(df)}")
print("\n **********************************")
    
# Sélection et renommage des colonnes nécessaires
df_filtered = df[['geonameid', 'name', 'latitude', 'longitude']].copy()
df_filtered.columns = ['ID', 'location_name', 'lat', 'long']
    
print("✓ Colonnes sélectionnées et renommées:")
print(f"  - ID (identifiant)")
print(f"  - location_name (nom du lieu)")
print(f"  - lat (latitude)")
print(f"  - long (longitude)")
    
# Affichage des premières lignes
print("\n **********************************")
print("\nPremières lignes du dataset:")
print(df_filtered.head())
    
# Sauvegarde en CSV
csv_filename = "burkina_location.csv"
df_filtered.to_csv(csv_filename, index=False, encoding='utf-8')
print("\n **********************************")
print(f"✓ Données sauvegardées dans {csv_filename}")
    


 **********************************
 Nombre de lignes (lieux): 11958

 **********************************
✓ Colonnes sélectionnées et renommées:
  - ID (identifiant)
  - location_name (nom du lieu)
  - lat (latitude)
  - long (longitude)

 **********************************

Premières lignes du dataset:
        ID       location_name       lat     long
0  2282318              Pouéné   9.72908 -2.78660
1  2285251  Léraba Occidentale  10.28333 -5.11667
2  2287216           Kéléouoro   9.80748 -4.05023
3  2294066         White Volta   8.70194 -0.99056
4  2298457        Issana Bouga  10.91667 -1.18333

 **********************************
✓ Données sauvegardées dans burkina_location.csv


In [4]:
# 4. Exploration des localités du Burkina Faso
print("\n **********************************")

# Chargement des données géographiques
print("\n Chargement du fichier des localités...")
df_burkina = pd.read_csv("burkina_location.csv")
print(f"✔ Données chargées: {len(df_burkina)} localités disponibles")
print(df_burkina.shape)


# 4.1 Recherche de localités de Gounghin
print("\n **********************************")
print("\n 4.1 - Recherche des localités contenant 'Gounghin'")
gounghin_data = df_burkina[df_burkina['location_name'].str.contains('gounghin', case=False, na=False)]

print(f"\n {len(gounghin_data)} localités correspondant à 'Gounghin' :")
print(gounghin_data)
    
# Sauvegarde des résultats
gounghin_data.to_csv("gounghin.csv", index=False)
print("\n résultats enregistrés dans le fichier 'gounghin.csv'")


# 4.2 Filtrage par ordre alphabétique
print("\n **********************************")
print("\n 4.2 - Localités dont le nom commence par A à P")

def commence_par_A_a_P(nom):
    """Vérifie si le nom commence par une lettre entre A et P"""
    if pd.isna(nom) or not nom.strip():
        return False
    return 'A' <= nom[0].upper() <= 'P'

localites_a_p = df_burkina[df_burkina['location_name'].apply(commence_par_A_a_P)]
print(f"\n Statistiques : {len(localites_a_p)} localités (A-P) sur {len(df_burkina)} au total")
print("\nVoici quelques exemples :")
print(localites_a_p.head(10))
   
# 4.3 Latitude et longitude minimales
print("\n **********************************")
print("\n4.3 Coordonnées minimales")
min_lat = df_burkina['lat'].min()
min_long = df_burkina['long'].min()
min_lat_location = df_burkina[df_burkina['lat'] == min_lat]['location_name'].iloc[0]
min_long_location = df_burkina[df_burkina['long'] == min_long]['location_name'].iloc[0]

print(f"Latitude minimale: {min_lat:.4f} - Lieu: {min_lat_location}")
print(f"Longitude minimale: {min_long:.4f} - Lieu: {min_long_location}")

# 4.4 Lieux avec coordonnées spécifiques (lat >= 11 et lon <= 0.5)
print("\n4.4 Lieux avec lat >= 11 et long <= 0.5")
specific_coords = df_burkina[(df_burkina['lat'] >= 11) & (df_burkina['long'] <= 0.5)]
print(f"Nombre de lieux correspondants: {len(specific_coords)}")
if not specific_coords.empty:
    print("Lieux trouvés:")
    print(specific_coords[['location_name', 'lat', 'long']])
else:
    print("Aucun lieu trouvé avec ces critères")


 **********************************

 Chargement du fichier des localités...
✔ Données chargées: 11958 localités disponibles
(11958, 4)

 **********************************

 4.1 - Recherche des localités contenant 'Gounghin'

 10 localités correspondant à 'Gounghin' :
             ID                  location_name       lat     long
153     2353306                       Gounghin  12.06677 -1.42134
7269    2360473                       Gounghin  12.62488 -1.36398
10260   2570204                       Gounghin  12.31436 -1.37900
10746  10342749                       Gounghin  12.06667 -0.15000
10759  10629032             BICIAB // Gounghin  12.35921 -1.54273
10818  11257296            Gounghin Department  12.06671 -0.15484
10845  11900526                  Gounghin Nord  12.36120 -1.55055
10846  11900528  Zone Industrielle de Gounghin  12.36631 -1.54137
10852  11900619                   Gounghin Sud  12.35298 -1.54342
10866  11900680                       Gounghin  12.35895 -1.54442

 r

In [5]:

print("\n5. CRÉATION DU FICHIER EXCEL")
print("-" * 30)

# Vérification des types
if not isinstance(gounghin_data, pd.DataFrame):
    raise TypeError("gounghin_data n'est pas un DataFrame")
if not isinstance(localites_a_p, pd.DataFrame):
    raise TypeError("commence_par_A_a_P n'est pas un DataFrame")

wb = Workbook()
wb.remove(wb.active)  # Supprimer la feuille par défaut

# --- Feuille Gounghin ---
ws_gounghin = wb.create_sheet(title="Gounghin")
for r in dataframe_to_rows(gounghin_data, index=False, header=True):
    ws_gounghin.append(r)

# --- Feuille A_to_P ---
ws_a_to_p = wb.create_sheet(title="A_to_P")
for r in dataframe_to_rows(localites_a_p, index=False, header=True):
    ws_a_to_p.append(r)

# Sauvegarde classeur mini_projet
excel_filename = "mini_projet.xlsx"
wb.save(excel_filename)

print(f"* Fichier Excel créé: {excel_filename}")
print(f"  - Feuille 'Gounghin': {len(gounghin_data)} lignes")
print(f"  - Feuille 'A_to_P': {len(localites_a_p)} lignes")






5. CRÉATION DU FICHIER EXCEL
------------------------------
* Fichier Excel créé: mini_projet.xlsx
  - Feuille 'Gounghin': 10 lignes
  - Feuille 'A_to_P': 8306 lignes
